In [2]:
!pip install stegano

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 5.4 MB/s eta 0:00:00


In [29]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
import hashlib
from stegano import lsb
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.backends import default_backend
from google.colab import files
import io

# Generate RSA keys for encryption and decryption
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)
public_key = private_key.public_key()

# Define the main buttons for each action
button_embed = widgets.Button(description="(1) Embed Text in Image (Steganography)", layout=widgets.Layout(width="400px"))
button_retrieve = widgets.Button(description="(2) Retrieve Text from Image (Steganography)", layout=widgets.Layout(width="400px"))
button_hash = widgets.Button(description="(3) Generate Hash of Image", layout=widgets.Layout(width="400px"))
button_encrypt = widgets.Button(description="(4) Encrypt Message", layout=widgets.Layout(width="400px"))
button_decrypt = widgets.Button(description="(5) Decrypt Encrypted Message", layout=widgets.Layout(width="400px"))
button_exit = widgets.Button(description="(S) Exit Application", layout=widgets.Layout(width="400px"))

# Display the menu
display(button_embed, button_retrieve, button_hash, button_encrypt, button_decrypt, button_exit)

# Define an output area for displaying results and messages
display_area = widgets.Output()
message_text = widgets.Text(description="Message:")
encrypted_text_input = widgets.Text(description="Encrypted (hex):")

# Helper function to handle image uploads
def upload_image():
    display_area.clear_output()
    with display_area:
        print("\nPlease upload an image.")
    uploaded = files.upload()
    for filename in uploaded.keys():
        image = Image.open(io.BytesIO(uploaded[filename]))
        with display_area:
            print(f"\nImage '{filename}' uploaded successfully.")
        return image

# Action functions for each button
def embed_text_action(b):
    clear_output(wait=True)
    display(button_embed, button_retrieve, button_hash, button_encrypt, button_decrypt, button_exit, display_area)
    display_area.clear_output()

    with display_area:
        original_image = upload_image()
        display(message_text)

    def on_message_submit(change):
        message = message_text.value
        stego_image = lsb.hide(original_image, message)
        stego_filename = "stego_image.png"
        stego_image.save(stego_filename)
        display_area.clear_output()
        with display_area:
            print("\nText embedded in image. Download the stego image:")
        files.download(stego_filename)

    message_text.on_submit(on_message_submit)

def retrieve_text_action(b):
    clear_output(wait=True)
    display(button_embed, button_retrieve, button_hash, button_encrypt, button_decrypt, button_exit, display_area)
    display_area.clear_output()
    with display_area:
        stego_image = upload_image()
        message = lsb.reveal(stego_image)
        print(f"\nRetrieved message: {message}")

def hash_action(b):
    clear_output(wait=True)
    display(button_embed, button_retrieve, button_hash, button_encrypt, button_decrypt, button_exit, display_area)
    display_area.clear_output()
    with display_area:
        print("\nPlease upload an image to generate its hash.")
    uploaded = files.upload()
    for filename in uploaded.keys():
        image_data = uploaded[filename]
        image_hash = hashlib.sha256(image_data).hexdigest()
        with display_area:
            print(f"\nSHA-256 Hash of '{filename}':\n {image_hash}")

def encrypt_action(b):
    clear_output(wait=True)
    display(button_embed, button_retrieve, button_hash, button_encrypt, button_decrypt, button_exit, display_area)
    display_area.clear_output()
    display(message_text)

    def on_message_submit(change):
        message = message_text.value
        encrypted_message = public_key.encrypt(
            message.encode(),
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        display_area.clear_output()
        with display_area:
            print("\nEncrypted message (in hexadecimal):")
            print(encrypted_message.hex())

    message_text.on_submit(on_message_submit)

def decrypt_action(b):
    clear_output(wait=True)
    display(button_embed, button_retrieve, button_hash, button_encrypt, button_decrypt, button_exit, display_area)
    display_area.clear_output()
    display(encrypted_text_input)

    def on_encrypted_submit(change):
        try:
            encrypted_message = bytes.fromhex(encrypted_text_input.value)
            decrypted_message = private_key.decrypt(
                encrypted_message,
                padding.OAEP(
                    mgf=padding.MGF1(algorithm=hashes.SHA256()),
                    algorithm=hashes.SHA256(),
                    label=None
                )
            )
            display_area.clear_output()
            with display_area:
                print(f"\nDecrypted message: {decrypted_message.decode()}")
        except ValueError:
            display_area.clear_output()
            with display_area:
                print("\nInvalid input for decryption. Ensure the message is in hexadecimal format.")

    encrypted_text_input.on_submit(on_encrypted_submit)

def exit_action(b):
    clear_output(wait=True)
    display_area.clear_output()
    with display_area:
        print("\nExiting application.")

# Attach functions to buttons
button_embed.on_click(embed_text_action)
button_retrieve.on_click(retrieve_text_action)
button_hash.on_click(hash_action)
button_encrypt.on_click(encrypt_action)
button_decrypt.on_click(decrypt_action)
button_exit.on_click(exit_action)

# Display the output area for feedback
display(display_area)


Button(description='(1) Embed Text in Image (Steganography)', layout=Layout(width='400px'), style=ButtonStyle(…

Button(description='(2) Retrieve Text from Image (Steganography)', layout=Layout(width='400px'), style=ButtonS…

Button(description='(3) Generate Hash of Image', layout=Layout(width='400px'), style=ButtonStyle())

Button(description='(4) Encrypt Message', layout=Layout(width='400px'), style=ButtonStyle())

Button(description='(5) Decrypt Encrypted Message', layout=Layout(width='400px'), style=ButtonStyle())

Button(description='(S) Exit Application', layout=Layout(width='400px'), style=ButtonStyle())

Output()

Text(value='5b11756c21f48140e0320e648176d8c201d67864ee30b9a3cb192fcf61a43cc491ddd60f1fb14b2f5ca37f637ca3846588…